In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import inout
from tqdm import tqdm_notebook as tqdm

In [2]:
df = pd.read_csv('e_high_bonus.in', sep=' ')
df.columns = ['start_row', 'start_col', 'end_row', 'end_col', 'earliest_start', 'latest_end']

with open('e_high_bonus.in') as f:
    first_line = re.sub(r'\n', '', f.readline())
    
variables = dict(zip(['n_rows', 'n_cols', 'n_cars', 'n_rides', 'bonus', 'steps'], [int(st) for st in first_line.split(" ")]))

In [3]:
variables

{'bonus': 1000,
 'n_cars': 350,
 'n_cols': 2000,
 'n_rides': 10000,
 'n_rows': 1500,
 'steps': 150000}

In [4]:
cars = [[[0, 0], 0]]*variables['n_cars']

In [5]:
solutions = [[]]*variables['n_cars']

In [6]:
def car_is_free(car):
    return car[1] <= 0

In [7]:
df.head()

,start_row,start_col,end_row,end_col,earliest_start,latest_end
0,295,1043,796,1955,38684,51050
1,434,467,200,833,89213,91692
2,1473,1815,1321,810,17720,26914
3,1210,150,1287,12,23916,24939
4,814,563,1341,419,21032,26079


In [8]:
df.iloc[1]

start_row           434
start_col           467
end_row             200
end_col             833
earliest_start    89213
latest_end        91692
Name: 1, dtype: int64

In [9]:
def dist(x, y):
    return abs(x[0] - y[0]) + abs(x[1] - y[1])

def gain_ride(start_pos, finish_pos, early_start, lastest_finish, current_time, bonus):
    pass

def gain_car(pos_car, start_pos, finish_pos, early_start, lastest_finish, current_time, bonus):
    """
    Return : (gain, time_needed) a tupple that is composed as first position of the 
    gain you obtain by taking this ride, and at second position the time in number of step
    until when the car will be free again. If gain is > 0 the car will be awalible at finish_pos after time_needed steps. 
    """
    dist2start = dist(pos_car, start_pos)
    dist_ride = dist(start_pos, finish_pos)
    
    if (current_time + dist2start + dist_ride) > lastest_finish:
        return (0, 0) #its impossible for this car to arrive on time at the finish
    
    gain = dist_ride #the reward is equal to the distance from start to end
    
    if (current_time + dist2start) <= early_start:
        #when on time at the begining we recieve a bonus
        gain+= bonus
    
    #Dont need to check if the ride is feasable, we assume it is!!!
    
    return (gain, max(dist2start, early_start - current_time) + dist_ride)

In [ ]:
df = df.sort_values('earliest_start')

In [ ]:
has_break = False
for t in tqdm(range(variables['steps'])):
    df = df[df.earliest_start >= t]
    if has_break:
        break
    for idx, car in enumerate(cars):
        if car_is_free(car):
            #get best ride
            max_gain = -1
            index = -1
            for r in df[0:100].iterrows():
                #print(r)
                gain, delta = gain_car(car[0], (r[1].start_row, r[1].start_col), (r[1].end_row, r[1].end_col), r[1].earliest_start, r[1].latest_end, t, variables['bonus'])
                if gain > max_gain:
                    max_gain = gain
                    index = r[0]
            #print(index)
            #print("lenght : " + str(len(df)))
            r = df.loc[index]
            task_taken, time_needed = gain_car(car[0], (r.start_row, r.start_col), (r.end_row, r.end_col), r.earliest_start, r.latest_end, t, variables['bonus'])
            car[1] = time_needed
            solutions[idx] = solutions[idx] + [index]
            df = df.drop(index)
            if len(df) <= 0:
                print("jesds")
                has_break = True
                break
        else:
            car[1] -= 1

A Jupyter Widget

In [ ]:
#solutions[0] = solutions[0] + [10]

In [ ]:
solutions

In [ ]:
inout.write_solution(solutions)